# Financial Instruments - Homework 5

## 1. Multiperiod Binomial Tree

**1.1 Replicating Portfolio**

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import plotly.graph_objects as go

# Parameters
S0 = 100.0
u = 1.1
d = 1/u
r = 0.05
K = 100.0

# Risk Neutral Prob
p = (1 + r - d) / (u - d)

# Prices at t=2
Suu = S0 * u**2
Sud = S0 * u * d
Sdd = S0 * d**2

# Payoffs
Cuu = max(Suu - K, 0)
Cud = max(Sud - K, 0)
Cdd = max(Sdd - K, 0)

# t=1 Values
Vu = (p * Cuu + (1 - p) * Cud) / (1 + r)
Vd = (p * Cud + (1 - p) * Cdd) / (1 + r)

# t=0 Value
V0 = (p * Vu + (1 - p) * Vd) / (1 + r)

print(f"Option Price V0: {V0:.4f}")

# Delta and B at t=0
Su = S0 * u
Sd = S0 * d
Delta_0 = (Vu - Vd) / (Su - Sd)
B_0 = (Vu - Delta_0 * Su) / (1 + r)

print(f"Delta_0: {Delta_0:.4f}")
print(f"B_0: {B_0:.4f}")

## 2. Black-Scholes (Verotende)

**Call and Put Prices**

In [ ]:
S = 42.0
K_bs = 40.0
T_bs = 0.5
sigma = 0.2
r_bs = 0.1

d1 = (np.log(S/K_bs) + (r_bs + 0.5 * sigma**2) * T_bs) / (sigma * np.sqrt(T_bs))
d2 = d1 - sigma * np.sqrt(T_bs)

Call = S * norm.cdf(d1) - K_bs * np.exp(-r_bs * T_bs) * norm.cdf(d2)
Put = K_bs * np.exp(-r_bs * T_bs) * norm.cdf(-d2) - S * norm.cdf(-d1)

print(f"Call: {Call:.4f}")
print(f"Put: {Put:.4f}")

# Visualization
S_range = np.linspace(20, 70, 100)
d1_r = (np.log(S_range/K_bs) + (r_bs + 0.5 * sigma**2) * T_bs) / (sigma * np.sqrt(T_bs))
d2_r = d1_r - sigma * np.sqrt(T_bs)
C_r = S_range * norm.cdf(d1_r) - K_bs * np.exp(-r_bs * T_bs) * norm.cdf(d2_r)

fig = go.Figure()
fig.add_trace(go.Scatter(x=S_range, y=C_r, name='Call Price'))
fig.update_layout(title='Call Option Convexity', xaxis_title='Stock Price', yaxis_title='Price')
fig.show()

## 3. Binomial Tree Convergence

In [ ]:
steps = [2, 5, 10, 25, 50, 125, 250]
results = []
for N in steps:
    dt = T_bs / N
    u_b = np.exp(sigma * np.sqrt(dt))
    d_b = 1 / u_b
    p_b = (np.exp(r_bs * dt) - d_b) / (u_b - d_b)
    
    j = np.arange(0, N + 1)
    ST = S * (u_b ** j) * (d_b ** (N - j))
    C = np.maximum(ST - K_bs, 0)
    
    for i in range(N - 1, -1, -1):
        C = np.exp(-r_bs * dt) * (p_b * C[1:] + (1 - p_b) * C[:-1])
    
    results.append({'N': N, 'Price': C[0]})

pd.DataFrame(results)